### AirBnB Price Prediction
- C14210085 Vincent Carel
- C14210109 Ella Arminta
- C14210225 Florentina Audrina

## Setting up environment

In [37]:
# https://brilliantprogrammer.medium.com/pyspark-random-forest-regression-machine-learning-a-practical-approach-part-7-459ffff42b8b
#importing required libs
import pyspark
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler,VectorIndexer, StringIndexer
from pyspark.ml import Pipeline
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql import SparkSession

from pyspark.sql.window import Window
from pyspark.sql.functions import col, row_number
from pyspark.sql.functions import coalesce

In [2]:
spark = (SparkSession.builder
         .appName('myApp')
         .master('local[*]')
         .getOrCreate()
        )
sc = spark.sparkContext 

### Reading the data

In [59]:
df = spark.read.format('csv').options(header=True, inferSchema=True, sep=',', quote='"', escape='"').load('train.csv')
df = df.select(['log_price','property_type','room_type','bathrooms','accommodates'])
df.show(truncate=True)
df.printSchema()

+------------------+-------------+---------------+---------+------------+
|         log_price|property_type|      room_type|bathrooms|accommodates|
+------------------+-------------+---------------+---------+------------+
| 5.010635294096256|    Apartment|Entire home/apt|      1.0|           3|
|5.1298987149230735|    Apartment|Entire home/apt|      1.0|           7|
| 4.976733742420574|    Apartment|Entire home/apt|      1.0|           5|
| 6.620073206530356|        House|Entire home/apt|      1.0|           4|
|  4.74493212836325|    Apartment|Entire home/apt|      1.0|           2|
| 4.442651256490317|    Apartment|   Private room|      1.0|           2|
|4.4188406077965965|    Apartment|Entire home/apt|      1.0|           3|
| 4.787491742782046|  Condominium|Entire home/apt|      1.0|           2|
| 4.787491742782046|        House|   Private room|      1.0|           2|
|  3.58351893845611|        House|   Private room|      1.0|           2|
| 4.605170185988092|    Apartment|   P

### Cleaning Data - Replace Null Value

In [60]:
# Checking for null or nan type values in our columns
from pyspark.sql.functions import col, count, isnan, when

df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+---------+-------------+---------+---------+------------+
|log_price|property_type|room_type|bathrooms|accommodates|
+---------+-------------+---------+---------+------------+
|        0|            0|        2|      202|           2|
+---------+-------------+---------+---------+------------+



In [61]:
# Drop table where room_type is null
df.filter(col("room_type").isNull()).show()
df_cleaned = df.dropna(subset=["room_type"])
df_cleaned.select([count(when(col(c).isNull(), c)).alias(c) for c in df_cleaned.columns]).show()

+---------+-------------+---------+---------+------------+
|log_price|property_type|room_type|bathrooms|accommodates|
+---------+-------------+---------+---------+------------+
|      1.0|          1.0|     NULL|     NULL|        NULL|
|      1.0|          1.0|     NULL|     NULL|        NULL|
+---------+-------------+---------+---------+------------+

+---------+-------------+---------+---------+------------+
|log_price|property_type|room_type|bathrooms|accommodates|
+---------+-------------+---------+---------+------------+
|        0|            0|        0|      200|           0|
+---------+-------------+---------+---------+------------+



In [62]:
# find the number of bathrooms based on property_type, room_type, and accommodates
bathrooms_book = df_cleaned.groupBy('property_type', 'room_type', 'bathrooms', 'accommodates').agg(count('*').alias('count'))
bathrooms_book.show()
# Window specification to order by count descending
window_spec = Window.partitionBy('property_type', 'room_type', 'accommodates').orderBy(col('count').desc())
# Add row number to each row within the partition
bathrooms_book_with_row_num = bathrooms_book.withColumn('row_number', row_number().over(window_spec))
# Filter to get only the rows with row number 1 (i.e., the most frequent bathrooms)
most_frequent_bathrooms = bathrooms_book_with_row_num.filter(col('row_number') == 1).drop('count', 'row_number')
# Show the result
most_frequent_bathrooms = most_frequent_bathrooms.withColumnRenamed('bathrooms', 'bathrooms_standard')
most_frequent_bathrooms.show()

+---------------+---------------+---------+------------+-----+
|  property_type|      room_type|bathrooms|accommodates|count|
+---------------+---------------+---------+------------+-----+
|      Timeshare|Entire home/apt|      1.0|           4|   31|
|           Loft|    Shared room|      1.5|           1|    2|
|          House|Entire home/apt|      5.0|          10|   10|
|     Guesthouse|Entire home/apt|      1.0|           4|   78|
|      Townhouse|Entire home/apt|      4.0|          14|    3|
|    Guest suite|Entire home/apt|      1.0|           4|   14|
|      Townhouse|   Private room|      1.0|           2|  370|
|      Apartment|Entire home/apt|      2.0|          10|  110|
|          House|Entire home/apt|      2.0|           6|  485|
|    Condominium|Entire home/apt|      2.5|          13|    1|
|          House|   Private room|      2.5|           4|   19|
|      Apartment|   Private room|      4.0|           4|    1|
|      Apartment|Entire home/apt|      1.0|          13

In [63]:
# replace the null value bathrooms based on bathrooms_book
# Join df_cleaned with most_frequent_bathrooms to get the most frequent bathrooms per group
df_with_most_frequent_bathrooms = df_cleaned.join(most_frequent_bathrooms, 
                                                  on=['property_type', 'room_type', 'accommodates'], 
                                                  how='left')
df_with_most_frequent_bathrooms.show()
# Replace null values in the bathrooms column
df_final = df_with_most_frequent_bathrooms.withColumn('bathrooms', 
                                                      coalesce(df_with_most_frequent_bathrooms['bathrooms_standard'],
                                                               df_with_most_frequent_bathrooms['bathrooms_standard']))

# Drop the extra bathrooms column used for replacement
df_final = df_final.drop(df_with_most_frequent_bathrooms['bathrooms_standard'])

# Show the final DataFrame
df_final.show()

+-------------+---------------+------------+------------------+---------+------------------+
|property_type|      room_type|accommodates|         log_price|bathrooms|bathrooms_standard|
+-------------+---------------+------------+------------------+---------+------------------+
|    Apartment|Entire home/apt|           6| 5.298317366548036|      1.5|               1.0|
|    Apartment|   Private room|           2| 4.442651256490317|      1.0|               1.0|
|    Apartment|   Private room|           2| 4.605170185988092|      1.0|               1.0|
|    Apartment|   Private room|           2| 4.248495242049359|      1.0|               1.0|
|    Apartment|   Private room|           2|4.0943445622220995|      1.5|               1.0|
|    Apartment|Entire home/apt|           5| 4.976733742420574|      1.0|               1.0|
|    Townhouse|   Private room|           2|  4.59511985013459|      2.0|               1.0|
|    Apartment|Entire home/apt|           2|  4.74493212836325|      1

In [64]:
df_final.filter(col("bathrooms").isNull()).show()
df_final = df_final.dropna(subset=["bathrooms"])
df_final.select([count(when(col(c).isNull(), c)).alias(c) for c in df_cleaned.columns]).show()
df_final.filter(col("bathrooms").isNull()).show()

+-------------+-----------+------------+-----------------+---------+
|property_type|  room_type|accommodates|        log_price|bathrooms|
+-------------+-----------+------------+-----------------+---------+
|Parking Space|Shared room|           1|4.605170185988092|     NULL|
+-------------+-----------+------------+-----------------+---------+

+---------+-------------+---------+---------+------------+
|log_price|property_type|room_type|bathrooms|accommodates|
+---------+-------------+---------+---------+------------+
|        0|            0|        0|        0|           0|
+---------+-------------+---------+---------+------------+

+-------------+---------+------------+---------+---------+
|property_type|room_type|accommodates|log_price|bathrooms|
+-------------+---------+------------+---------+---------+
+-------------+---------+------------+---------+---------+



### Preprocess Data - Categorical -> Numerical

In [65]:
property_type_indexer = StringIndexer(inputCol="property_type", outputCol="property_type_index")
room_type_indexer = StringIndexer(inputCol="room_type", outputCol="room_type_index")

# Fit and transform the StringIndexers
indexer_model = property_type_indexer.fit(df_final)
df_final = indexer_model.transform(df_final)

indexer_model = room_type_indexer.fit(df_final)
df_final = indexer_model.transform(df_final)

# Show the DataFrame with categorical variables
df_final.show()
df_final.groupBy('property_type_index', 'room_type_index').agg(count('*').alias('count')).show()

+-------------+---------------+------------+------------------+---------+-------------------+---------------+
|property_type|      room_type|accommodates|         log_price|bathrooms|property_type_index|room_type_index|
+-------------+---------------+------------+------------------+---------+-------------------+---------------+
|    Apartment|Entire home/apt|           1| 4.605170185988092|      1.0|                0.0|            0.0|
|    Apartment|Entire home/apt|           1| 5.135798437050261|      1.0|                0.0|            0.0|
|    Apartment|Entire home/apt|           1| 4.836281906951479|      1.0|                0.0|            0.0|
|    Apartment|Entire home/apt|           1|5.1929568508902095|      1.0|                0.0|            0.0|
|    Apartment|Entire home/apt|           1|5.0751738152338275|      1.0|                0.0|            0.0|
|    Apartment|Entire home/apt|           1| 4.174387269895637|      1.0|                0.0|            0.0|
|    Apart

### Tuning Data

In [ ]:
## masih blm pilih parameter dan Tuning data

### Assemble all our features to one vector


In [107]:
features = df_final.select(['property_type_index','room_type_index','bathrooms','accommodates'])
features.filter((col("property_type_index") != 0.0) & (col("room_type_index") != 0.0)).show()

+-------------------+---------------+---------+------------+
|property_type_index|room_type_index|bathrooms|accommodates|
+-------------------+---------------+---------+------------+
|                7.0|            1.0|      1.0|           1|
|                7.0|            1.0|      1.0|           1|
|                7.0|            1.0|      1.0|           1|
|                7.0|            1.0|      1.0|           1|
|                7.0|            1.0|      1.0|           1|
|                7.0|            1.0|      1.0|           1|
|                7.0|            1.0|      1.0|           1|
|                7.0|            1.0|      1.0|           1|
|                7.0|            1.0|      1.0|           1|
|                7.0|            1.0|      1.0|           1|
|                7.0|            1.0|      1.0|           1|
|                7.0|            1.0|      1.0|           1|
|                7.0|            1.0|      1.0|           1|
|                7.0|   

In [108]:
assembler = VectorAssembler( inputCols=features.columns,outputCol="features")
output = assembler.transform(df_final)

### Random Forest Regressor
Class for indexing categorical feature columns in a dataset of Vector.


In [109]:
featureIndexer =VectorIndexer(inputCol="features", outputCol="indexedFeatures").fit(output)

featureIndexer = featureIndexer.transform(output)
featureIndexer
# DataFrame[features: vector, Chance of Admit: float, indexedFeatures: vector]

new_indexed_data = featureIndexer.select("indexedFeatures", "log_price")
new_indexed_data.show()

+-----------------+------------------+
|  indexedFeatures|         log_price|
+-----------------+------------------+
|[0.0,0.0,2.0,0.0]| 4.605170185988092|
|[0.0,0.0,2.0,0.0]| 5.135798437050261|
|[0.0,0.0,2.0,0.0]| 4.836281906951479|
|[0.0,0.0,2.0,0.0]|5.1929568508902095|
|[0.0,0.0,2.0,0.0]|5.0751738152338275|
|[0.0,0.0,2.0,0.0]| 4.174387269895637|
|[0.0,0.0,2.0,0.0]| 3.912023005428147|
|[0.0,0.0,2.0,0.0]| 5.342334251964812|
|[0.0,0.0,2.0,0.0]|  4.74493212836325|
|[0.0,0.0,2.0,0.0]| 4.605170185988092|
|[0.0,0.0,2.0,0.0]|4.0943445622220995|
|[0.0,0.0,2.0,0.0]| 4.787491742782046|
|[0.0,0.0,2.0,0.0]| 5.634789603169249|
|[0.0,0.0,2.0,0.0]| 4.007333185232471|
|[0.0,0.0,2.0,0.0]| 4.219507705176107|
|[0.0,0.0,2.0,0.0]| 4.553876891600541|
|[0.0,0.0,2.0,0.0]|  4.48863636973214|
|[0.0,0.0,2.0,0.0]|4.0943445622220995|
|[0.0,0.0,2.0,0.0]| 4.442651256490317|
|[0.0,0.0,2.0,0.0]| 5.204006687076795|
+-----------------+------------------+
only showing top 20 rows



### Split the Data

In [110]:
training, test = new_indexed_data.randomSplit([0.7, 0.3])

training.show()
test.show()

+-----------------+------------------+
|  indexedFeatures|         log_price|
+-----------------+------------------+
|[0.0,0.0,2.0,0.0]| 3.218875824868201|
|[0.0,0.0,2.0,0.0]| 3.688879454113936|
|[0.0,0.0,2.0,0.0]| 3.688879454113936|
|[0.0,0.0,2.0,0.0]| 3.688879454113936|
|[0.0,0.0,2.0,0.0]|  3.80666248977032|
|[0.0,0.0,2.0,0.0]|  3.80666248977032|
|[0.0,0.0,2.0,0.0]|  3.80666248977032|
|[0.0,0.0,2.0,0.0]|  3.80666248977032|
|[0.0,0.0,2.0,0.0]|  3.80666248977032|
|[0.0,0.0,2.0,0.0]|  3.80666248977032|
|[0.0,0.0,2.0,0.0]|  3.80666248977032|
|[0.0,0.0,2.0,0.0]|3.8918202981106265|
|[0.0,0.0,2.0,0.0]|3.8918202981106265|
|[0.0,0.0,2.0,0.0]| 3.912023005428147|
|[0.0,0.0,2.0,0.0]| 3.912023005428147|
|[0.0,0.0,2.0,0.0]| 3.912023005428147|
|[0.0,0.0,2.0,0.0]| 3.912023005428147|
|[0.0,0.0,2.0,0.0]| 3.912023005428147|
|[0.0,0.0,2.0,0.0]| 3.912023005428147|
|[0.0,0.0,2.0,0.0]| 3.912023005428147|
+-----------------+------------------+
only showing top 20 rows

+-----------------+------------------+

### Apply Random Regressor

In [111]:
random_forest_reg = RandomForestRegressor(featuresCol="indexedFeatures",labelCol="log_price",maxBins=40)

random_forest_reg

RandomForestRegressor_385bb840b11b

### Train Model

In [112]:
model = random_forest_reg.fit(training)

model

RandomForestRegressionModel: uid=RandomForestRegressor_385bb840b11b, numTrees=20, numFeatures=4

### Making Prediction on test Data

In [113]:
predictions = model.transform(test)

predictions.show(400, truncate=False)

+-----------------+------------------+-----------------+
|indexedFeatures  |log_price         |prediction       |
+-----------------+------------------+-----------------+
|[0.0,0.0,2.0,0.0]|3.332204510175204 |4.843590088879641|
|[0.0,0.0,2.0,0.0]|3.6635616461296463|4.843590088879641|
|[0.0,0.0,2.0,0.0]|3.80666248977032  |4.843590088879641|
|[0.0,0.0,2.0,0.0]|3.912023005428147 |4.843590088879641|
|[0.0,0.0,2.0,0.0]|3.912023005428147 |4.843590088879641|
|[0.0,0.0,2.0,0.0]|3.912023005428147 |4.843590088879641|
|[0.0,0.0,2.0,0.0]|3.951243718581428 |4.843590088879641|
|[0.0,0.0,2.0,0.0]|3.970291913552122 |4.843590088879641|
|[0.0,0.0,2.0,0.0]|4.007333185232471 |4.843590088879641|
|[0.0,0.0,2.0,0.0]|4.007333185232471 |4.843590088879641|
|[0.0,0.0,2.0,0.0]|4.007333185232471 |4.843590088879641|
|[0.0,0.0,2.0,0.0]|4.060443010546419 |4.843590088879641|
|[0.0,0.0,2.0,0.0]|4.0943445622220995|4.843590088879641|
|[0.0,0.0,2.0,0.0]|4.0943445622220995|4.843590088879641|
|[0.0,0.0,2.0,0.0]|4.0943445622

In [117]:
# Evaluate the model
evaluator = RegressionEvaluator(labelCol="log_price", predictionCol="prediction", metricName="rmse")
print ("Root Mean Squared Error (RMSE) on test data = ",evaluator.evaluate(predictions), "(mendekati 0 lebih akurat)")

evaluator = RegressionEvaluator(labelCol="log_price",predictionCol="prediction", metricName="r2")
print("R Squared (R2) on test data =", evaluator.evaluate(predictions), "(mendekati 1 lebih akurat)")

# Evaluate MAE
evaluator_mae = RegressionEvaluator(labelCol="log_price", predictionCol="prediction", metricName="mae")
mae = evaluator_mae.evaluate(predictions)
print("Mean Absolute Error (MAE) on test data = ", mae , "(mendekati 0 lebih akurat)")

# Evaluate MSE
evaluator_mse = RegressionEvaluator(labelCol="log_price", predictionCol="prediction", metricName="mse")
mse = evaluator_mse.evaluate(predictions)
print("Mean Squared Error (MSE) on test data = ", mse , "(mendekati 0 lebih akurat)")

Root Mean Squared Error (RMSE) on test data =  0.508529590340666 (mendekati 0 lebih akurat)
R Squared (R2) on test data = 0.5017359407907721 (mendekati 1 lebih akurat)
Mean Absolute Error (MAE) on test data =  0.3858457040680029 (mendekati 0 lebih akurat)
Mean Squared Error (MSE) on test data =  0.2586023442520456 (mendekati 0 lebih akurat)
